In [1]:
import pandas as pd
from os import chdir
import subprocess
import glob

# local import (Hirotaka's myfunctions.py script)
import myfunctions as myfunc

In [2]:
!mkdir temp
chdir('temp')

mkdir: cannot create directory 'temp': File exists


In [3]:
# Get the id file, covariate file (all European)
wgsfolder = '/data/CARD/PD/GENOMES/august19/genotypes'
idfile = '_keep.id'
demog = pd.read_csv('/data/LNG/iwakih2/ampPD/data1_out/PP_PD_BF_HB_LC_demog.csv')
euro = pd.read_csv(f'{wgsfolder}/PCA_filtered_europeans.txt', sep=' ',
                  header=None, names=['FID', 'IID'])
df = pd.merge(demog, euro, left_on='WGSID', right_on='IID')
df['FID'] = df.WGSID
df['IID'] = df.WGSID
df.RECRUIT=['CTR' if i=='HC' else i for i in df.RECRUIT]
df = df[df.RECRUIT.isin(['CTR', 'PD'])]
df = df[df.RECRUIT==df.LASTDIAG]
df['pheno'] = [1 if i=='PD' else 0 for i in df.RECRUIT]
df['AAD'] = df.AADi.where(df.pheno==0, other=df.AAOi)
df['AGE'] = df.AGEatBL.where(df.pheno==0, other=df.AAD)

df[['FID', 'IID', 'pheno']].to_csv(idfile, index=False, sep='\t', header=None)
df[['FID', 'IID', 'AGE', 'FEMALE']].to_csv('_qcovar.txt', index=False, sep='\t', header=None)

In [25]:
df.head()

,Unnamed: 0,DATASET,STUDY,SOURCE,clinID,WGSID,AMPID,AMPseq,nxID,AGEatBL,...,RECRUIT,AADi,AAOi,LASTDIAG,comment,FID,IID,pheno,AAD,AGE
0,1,BF,BIOFIND,LONI_BIOFIND,1001,BF-1001,BF-1001,1.0,1001/1001,54.989733,...,CTR,NaN,NaN,CTR,NaN,BF-1001,BF-1001,0,NaN,54.989733
1,2,BF,BIOFIND,LONI_BIOFIND,1002,BF-1002,BF-1002,1.0,1002/1002,65.993155,...,PD,60.988364,NaN,PD,NaN,BF-1002,BF-1002,1,NaN,NaN
2,3,BF,BIOFIND,LONI_BIOFIND,1003,BF-1003,BF-1003,1.0,1003/1003,60.993840,...,PD,55.991786,NaN,PD,NaN,BF-1003,BF-1003,1,NaN,NaN
3,4,BF,BIOFIND,LONI_BIOFIND,1004,BF-1004,BF-1004,1.0,1004/1004,61.995893,...,PD,55.014374,NaN,PD,NaN,BF-1004,BF-1004,1,NaN,NaN
4,5,BF,BIOFIND,LONI_BIOFIND,1005,BF-1005,BF-1005,1.0,NaN,60.999316,...,CTR,NaN,NaN,CTR,NaN,BF-1005,BF-1005,0,NaN,60.999316


In [ ]:
# for chrnum in range(1,23):
#     pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
#     out = f'{wgsfolder}/merge/pd.june2019.chr{chrnum}.sqc'
#     plinkcmd = f'\
#  plink2 --pfile {pfile}\
#  --make-bed\
#  --out {out}'
#    myfunc.shell_do(plinkcmd)

In [ ]:
# with open('_mergelist.txt', 'w') as f:
#     for chrnum in range(1,23):
#         bfile = f'{wgsfolder}/merge/pd.june2019.chr{chrnum}.sqc'
#         f.write(f'{bfile}\n')
    

In [ ]:
# mergecmd = f'\
# plink --merge-list _mergelist.txt\
#  --make-bed\
#  --out {wgsfolder}/merge/pd.june2019.sqc'
# with open('_merge_batch_script.sh', 'w') as f:
#     f.write('#!/bin/bash\n')
#     f.write('module load plink\n')
#     f.write(f'{mergecmd}')

In [ ]:
# !sbatch _merge_batch_script.sh --cpus-per-task=128 --mem=100g

In [ ]:
# used this to generate pca for chr1 just for testing purposes-- prune and merge all chrs first then run PCA
with open('_pca.swarm', 'w') as f:
    for chrnum in range(1,2):
        pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
        out_pcs = f'chr{chrnum}.pca'
        plink_pca_cmd = f'\
plink2 --pfile {pfile}\
 --pca 50\
 --out {out_pcs}\n'
        
        f.write(plink_pca_cmd)
        print(plink_pca_cmd)

In [ ]:
# !swarm -f _pca.swarm -g 64 --time=10:00:00 -t 20 --logdir swarm --module plink/2.0_alpha_1_final --partition=norm

In [ ]:
with open('_script1.sh', 'w') as f:
    for chrnum in range(1,):
        pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
        freqfile = f'freq.{chrnum}'
        plinkcmd = f"\
plink2 --pfile {pfile}\
 --keep {idfile} \
 --freq \
 --out {freqfile}\n"
        f.write(plinkcmd)
#         myfunc.shell_do(plinkcmd)
print(f'{plinkcmd}')

In [5]:
from dask.distributed import Client, progress
# client should match the qued number of threads
# client = Client(processes=False, threads_per_worker=2,
#                 n_workers=2, memory_limit='30GB')
# import dask
import dask.dataframe as dd

#try testing fewer maf bins
def MAFbin(x):
    if x>0:
        if x<0.001:
            return(1)
        elif x<0.01:
            return(2)
        elif x<0.1:
            return(3)
        elif x<0.25:
            return(4)
        elif x<0.5:
            return(5)
        else:
            return(6)
    else:
        return(0)



# def MAFbin(x):
#     if x>0:
#         if x<0.001:
#             return(1)
#         elif x<0.01:
#             return(2)
#         elif x<0.1:
#             return(3)
#         elif x<0.2:
#             return(4)
#         elif x<0.3:
#             return(5)
#         elif x<0.4:
#             return(6)
#         else:
#             return(7)
#     else:
#         return(0)



In [ ]:
client.close()

In [78]:
total_counts_df = pd.DataFrame(index=[0,1,2,3,4,5,6])

for chrnum in range(1,23):
    freqfile = f'freq.{chrnum}'

    df = dd.read_csv(f'{freqfile}.afreq', sep='\t')
    df['MAF'] = df.ALT_FREQS.where(cond=df.ALT_FREQS<=0.5, other=1-df.ALT_FREQS)
    df['Group'] = df.MAF.apply(MAFbin, meta=('MAF', 'float64'))
    for i in range(1,7):
        df.loc[df.Group==i, ['ID']].to_csv(
            f'{freqfile}.mafbin{i}', index=False, single_file=True)
    print(f'chr{chrnum} finished')
    total_counts_df[chrnum] = pd.DataFrame(df.groupby('Group').ID.count().compute())
#     total_counts_df.merge(counts_df, left_index=True)

chr1 finished
chr2 finished
chr3 finished
chr4 finished
chr5 finished
chr6 finished
chr7 finished
chr8 finished
chr9 finished
chr10 finished
chr11 finished
chr12 finished
chr13 finished
chr14 finished
chr15 finished
chr16 finished
chr17 finished
chr18 finished
chr19 finished
chr20 finished
chr21 finished
chr22 finished


In [10]:
# # add chr names
chrs = ['chr' + str(i) for i in range(1,23)]
total_counts_df.columns = chrs

# add bins
bins = ['0.000', '<0.001', '<0.01', '<0.1', '<0.25', '<0.5', '=0.5']
total_counts_df['bin'] = bins
total_counts_df.set_index('bin', inplace=True)
total_counts_df['total'] = total_counts_df.sum(axis=1)
total_counts_df.to_csv('_maf_bin_counts.txt', sep='\t')
total_counts_df

,chr1,chr2,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chr10,...,chr14,chr15,chr16,chr17,chr18,chr19,chr20,chr21,chr22,total
bin,,,,,,,,,,,,,,,,,,,,,
0.000,7266123,7721151,6307030,6076066,5706089,5343353,5185566,4906960,3988065,4322263,...,2846757,2646948,2996448,2639022,2500212,2053859,2062406,1204377,1297916,88786823
0.001,5239505,5383995,4319848,4162827,3898668,3634457,3630559,3380543,2774901,3034268,...,1949417,1821648,2154175,1910066,1715748,1553700,1471411,858014,973767,62034698
0.01,1050342,996780,791823,775054,719608,689139,697280,621412,526004,602308,...,370017,352293,422866,392798,328423,333077,293520,179951,212192,11892600
0.1,542322,563218,451356,466013,414715,431617,404283,358642,293287,349447,...,222821,197487,219241,198739,192450,173381,160200,97833,108977,6744502
0.25,253109,275704,233387,236875,209134,223369,195627,174538,143516,171258,...,107111,94303,101781,96200,92757,86375,77047,45324,47271,3307007
0.5,284120,299198,258502,268285,236372,249545,221131,196881,153119,183800,...,116246,105287,112279,104315,105037,87930,82546,52582,54507,3667396
>0.5,157,196,179,156,97,104,143,89,92,150,...,81,94,71,134,90,51,58,52,40,2381


In [37]:
freqfile = f'freq.21'

df = pd.read_csv(f'{freqfile}.afreq', sep='\t')
df['MAF'] = df.ALT_FREQS.where(cond=df.ALT_FREQS<=0.5, other=1-df.ALT_FREQS)


In [39]:
df['Group'] = df.MAF.apply(MAFbin)
df.head()

,#CHROM,ID,REF,ALT,ALT_FREQS,OBS_CT,MAF,Group
0,21,chr21:5030240:AC:A,AC,A,0.003916,5362,0.003916,2
1,21,chr21:5030578:C:T,C,T,0.000171,5844,0.000171,1
2,21,chr21:5030582:A:T,A,T,0.000171,5842,0.000171,1
3,21,chr21:5030588:T:C,T,C,0.000171,5850,0.000171,1
4,21,chr21:5030591:C:T,C,T,0.000000,5846,0.000000,0


In [44]:
df[df.Group == 6].head()

,#CHROM,ID,REF,ALT,ALT_FREQS,OBS_CT,MAF,Group
18587,21,chr21:6070564:TGTGGTTTGTGG:T,TGTGGTTTGTGG,T,0.5,4,0.5,6
37746,21,chr21:7327861:G:GAATCAGCTCTGTGATAT,G,GAATCAGCTCTGTGATAT,0.5,2,0.5,6
56368,21,chr21:8210561:C:CG,C,CG,0.5,8,0.5,6
165644,21,chr21:10274324:G:GTTCATACTGACCTGTCAAATGCGAACTC,G,GTTCATACTGACCTGTCAAATGCGAACTC,0.5,2,0.5,6
237702,21,chr21:13558463:T:C,T,C,0.5,4,0.5,6


In [14]:
with open('_script2.swarm', 'w') as f:
    for chrnum in range(1,23):
        pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
        freqfile = f'freq.{chrnum}'
        for Indx in range(1,7):
            mafbinIndx = f'mafbin{Indx}'
            mafbinfile =  f'{freqfile}.{mafbinIndx}'

            plinkcmd = f"\
plink2 --pfile {pfile}\
 --extract {mafbinfile}\
 --keep {idfile}\
 --make-bed\
 --out {mafbinfile}"
            gctacmd1 = f'\
gcta64 --bfile {mafbinfile}\
 --ld-score-region 10000\
 --out {mafbinfile}.ldbin\
 --thread-num 10'

            f.write(f'{plinkcmd} && {gctacmd1}\n')

In [15]:
# !swarm -f _script2.swarm --time=40:00:00 -g 64 -t 10 --logdir swarm --module GCTA,plink/2.0_alpha_1_final --partition=norm --devel

60772194


In [1]:
for chrnum in range(1,23):
    pass

In [11]:
# need to rerun for jobs that were killed
outputs = glob.glob('freq.*.mafbin*.ldbin.score.ld')
# len(glob.glob('*.ldbin.log'))
expected_out = ['freq.' + str(i) + '.mafbin' + str(j) + '.ldbin.score.ld' for i in range(1,23) for j in range(1,7)]
missing = [bed.replace('.ldbin.score.ld', '') for bed in list(set(expected_out) - set(outputs))]

with open('_rerunscript.swarm', 'w') as f:
    for mafbinfile in missing:
        gctacmd1 = f'\
gcta64 --bfile {mafbinfile}\
 --ld-score-region 10000\
 --out {mafbinfile}.ldbin\
 --thread-num 10'

        f.write(f'{gctacmd1}\n')

In [12]:
!swarm -f _rerunscript.swarm --time=40:00:00 -g 64 -t 10 --logdir swarm --module GCTA --partition=norm

60864905


In [9]:
for chrnum in range(1,23):
    pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
    freqfile = f'freq.{chrnum}'
    for Indx in range(1,7):
        mafbinIndx = f'mafbin{Indx}'
        mafbinfile =  f'{freqfile}.{mafbinIndx}'

        
        

        # Note this is the ldscore within the maf bin
        df = dd.read_csv(f'{mafbinfile}.ldbin.score.ld', sep=' ')
        ldscvec = df.ldscore_SNP.compute()
        ldscThres = ldscvec.median()
        df['Group'] = 1
        df['Group'] = df.Group.where(cond=df.ldscore_SNP<ldscThres, other=2)


        # Create ldbin and GRMs within the ldbin
        for i in range(1, 3):
            ldbinIndx = f'{mafbinfile}.ldbin{i}'
            df.loc[df.Group==i, ['SNP']].to_csv(
                f'{ldbinIndx}', index=False, single_file=True)
#             gctacmd2 = f'\
#             gcta64 --bfile {mafbinfile}\
#  --extract {ldbinIndx}\
#  --make-grm\
#  --out {ldbinIndx}'
#             myfunc.shell_do(gctacmd2)
            with open('multi_GRMs.txt', 'a') as f:
                f.write(f'{ldbinIndx}\n')
        # mafbin information save to mafbinInfo.csv
        with open('mafbinInfo.txt', 'a') as f:
            s = f'{chrnum}\t{mafbinIndx}\t{len(ldscvec)}\t{ldscvec.mean()}\t{ldscvec.median()}\t{ldscvec.std()}'
            f.write(f'{s}\n')

In [36]:
# create covar file per chrom and impute missing age (may want to consider dropping age column in this case (missing for more than half))
# impute "FEMALE" with median
for chrnum in range(1,2):
    pcs_df = pd.read_csv(f'chr{chrnum}.pca.eigenvec', sep='\t')
    pcs_df.rename(columns={'#FID':'FID'}, inplace=True)
    covar_df = df[['FID', 'IID', 'AGE', 'FEMALE']].merge(pcs_df, on=['FID','IID'])
    covar_df.drop(['AGE','FEMALE'], axis=1, inplace=True)
#     covar_df.AGE.fillna(covar_df.AGE.mean(), inplace=True)
#     covar_df.FEMALE.fillna(covar_df.FEMALE.median(),inplace=True)
#     print(f'dropping {covar_df[covar_df.AGE.isnull()].shape[0]} rows for missing age')
#     covar_df.dropna(how='any', inplace=True)
    covar_df.to_csv(f'chr{chrnum}.covar', index=False, sep=' ', header=None)
    print(f'wrote: chr{chrnum}.covar with {covar_df.shape[0]} samples')

wrote: chr1.covar with 3181 samples


In [37]:
covar_df.head()

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50
0,BF-1001,BF-1001,-0.002224,1.248830e-04,0.000250,0.000070,0.000019,-0.000564,-0.000007,0.000188,...,-0.000147,0.000114,0.000009,-0.000143,-0.000037,-0.000060,0.000007,-0.000028,-2.282550e-04,-0.000109
1,BF-1002,BF-1002,-0.001984,6.582490e-05,0.000315,-0.000068,-0.000034,-0.000240,0.000016,-0.000026,...,-0.000043,-0.000058,0.000241,0.000046,0.000180,-0.000051,0.000165,-0.000156,4.493480e-06,-0.000043
2,BF-1003,BF-1003,-0.001857,4.495790e-07,0.000419,0.000025,0.000042,-0.000988,0.000048,0.000014,...,0.000167,-0.000142,0.000356,-0.000201,-0.000010,-0.000192,-0.000138,-0.000226,2.619660e-07,-0.000085
3,BF-1004,BF-1004,-0.001708,2.793810e-05,0.000228,0.000200,-0.000046,0.000194,0.000041,0.000009,...,-0.000117,-0.000035,-0.000099,0.000146,0.000082,-0.000176,0.000135,-0.000264,-1.218550e-04,0.000196
4,BF-1005,BF-1005,-0.001882,1.249420e-04,0.000104,-0.000018,0.000012,0.000249,-0.000044,0.000015,...,0.000159,-0.000101,0.000182,0.000025,0.000052,0.000137,0.000033,-0.000025,-6.337450e-05,-0.000083


In [38]:
#--reml-no-lrt\
# --reml-inv-mtd 3\

gctacmd3 = f'\
gcta --reml\
 --mgrm multi_GRMs.txt\
 --pheno {idfile}\
 --qcovar chr1.pca.eigenvec\
 --reml-no-constrain\
 --thread-num 8\
 --out test'
myfunc.shell_do(gctacmd3)

# with open('_reml_script.sh', 'w') as f:
#     f.write('#!/bin/bash\n')
#     f.write(f'{gctacmd3}\n')

Executing: gcta --reml --mgrm multi_GRMs.txt --pheno _keep.id --qcovar chr1.pca.eigenvec --reml-no-constrain --thread-num 8 --out test


In [ ]:
!sbatch --mem=100g --cpus-per-task=64 _reml_script.sh

In [13]:
22*7*2

308

NameError: name 'covar_df' is not defined